<a href="https://colab.research.google.com/github/Jacobikinz/Project-Hummingbird/blob/master/Simple_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import matplotlib.pyplot as ply, numpy as np, pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
import zipfile 

#file = 'drive/My Drive/MC Villasenor - Computational Modeling/4th Quarter Project - Project Hummingbird/million-headlines.zip'

#with ZipFile(file, 'r') as zip:
# headlines = zip.open('abcnews-date-text.csv')
  
  
zip_ref = zipfile.ZipFile('/content/drive/My Drive/MC Villasenor - Computational Modeling/4th Quarter Project - Project Hummingbird/global-news-week.zip', 'r')
zip_ref.extractall('/content/drive/My Drive/Colab Notebooks/')
print("News imported.")

zip_ref.close()

News imported.


In [9]:
cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [10]:
ls

 abcnews-date-text.csv                  news-week-17aug24.csv
 companylist.csv                        news-week-18aug24.csv
 CompanyTrend.ipynb                     SentiWordNet_3.0.0.txt
 Data.zip                              'Simple Predictor.ipynb'
 ETFs/                                  Stocks/
 github.com/                           'Text Scraper.ipynb'
 hummingbird-239418-1317d3f83b56.json  'Training Finance Data.ipynb'
 NASDAQFun.ipynb                       'Yahoo Finance.ipynb'
 NewsAPI.ipynb


In [11]:
newsHeadlines = pd.read_csv('news-week-17aug24.csv', dtype={0:'str'})
print("News done.")
priceData = pd.read_csv('https://github.com/Jacobikinz/Project-Hummingbird/raw/master/Allstocks_THEDATA.csv')
print("Price done.")
sentiment = pd.read_csv('https://raw.githubusercontent.com/aesuli/SentiWordNet/master/data/SentiWordNet_3.0.0.txt', delimiter='\t', skiprows = 25)
print("Sentiment done.")

News done.
Price done.
Sentiment done.


##SENTIMENT ANALYSIS

Chunk-ing off top words: 

In [0]:
rawSort = sentiment.drop(['# POS', 'ID', 'Gloss'], axis = 1)

rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'#',value=r'')
rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'1',value=r'')
rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'2',value=r'')
rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'3',value=r'')
rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'4',value=r'')
rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'5',value=r'')
rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'6',value=r'')
rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'7',value=r'')
rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'8',value=r'')
rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'\_',value=r' ')
rawSort['SynsetTerms'].replace(regex=True,inplace=True,to_replace=r'\-',value=r' ')


positive = rawSort.sort_values(by='PosScore', ascending=False)
positive = positive.head(3000)

negative = rawSort.sort_values(by='NegScore', ascending=False)
negative = negative.head(3000)

rawSort["GenScore"] = rawSort["PosScore"]-rawSort["NegScore"]
rawSort = rawSort.reset_index()

In [49]:
rawSort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117660 entries, 0 to 117659
Data columns (total 4 columns):
PosScore       117659 non-null float64
NegScore       117659 non-null float64
SynsetTerms    117660 non-null object
GenScore       117659 non-null float64
dtypes: float64(3), object(1)
memory usage: 3.6+ MB


In [50]:
!pip install spacy

In [0]:
#https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

punctuations = string.punctuation

nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

parser = English()

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    return mytokens
  

class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

def clean_text(text):
    return text.strip().lower()
  
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [66]:
#https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

from sklearn.model_selection import train_test_split

X = rawSort['SynsetTerms']
ylabels = rawSort["GenScore"]

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2)

ValueError: ignored

In [67]:
#https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

pipe.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: ignored

In [0]:
scrubHeadlines = newsHeadlines.drop(['source_url', 'feed_code'], axis = 1)

scrubHeadlines['headline_text'].replace(regex=True,inplace=True,to_replace=r'\$',value=r'')
scrubHeadlines['headline_text'].replace(regex=True,inplace=True,to_replace=r'\'',value=r'')
scrubHeadlines['headline_text'].replace(regex=True,inplace=True,to_replace=r',',value=r'')

In [0]:
scrubHeadlines['publish_time'] = scrubHeadlines['publish_time'].str.slice(0,8)
groupedHeadlines = scrubHeadlines.groupby(['publish_time'])['headline_text'].apply(list).reset_index()
#''.join('headline_text')
#groupedHeadlines['headline_text'][0]

In [0]:
dataset = groupedHeadlines['headline_text'][0]

In [0]:
#!pip install google.cloud

#import os
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='hummingbird-239418-1317d3f83b56.json'

In [0]:
#def gc_sentiment(text):  
#    from google.cloud import language
#    
#    client = language.LanguageServiceClient()
#    document = language.types.Document(
#            content=text,
#            type=language.enums.Document.Type.PLAIN_TEXT)
#    annotations = client.analyze_sentiment(document=document)
#    score = annotations.document_sentiment.score
#    magnitude = annotations.document_sentiment.magnitude
#    return score, magnitude

In [0]:
#from tqdm import tqdm
#import pandas as pd
#gc_results = [gc_sentiment(row) for row in tqdm(dataset, ncols = 100)]
#gc_score, gc_magnitude = zip(*gc_results) # Unpacking the result into 2 lists
#gc = list(zip(dataset, gc_score, gc_magnitude))
#columns = ['text', 'score', 'magnitude']
#gc_df = pd.DataFrame(gc, columns = columns)

In [0]:
text = scrubHeadlines['headline_text'][0]